In [2]:
import os
os.chdir("../src")
from tool_preprocess_onehot import preprocesing_function_onehot
os.chdir("../data")
import pandas as pd
df = pd.read_csv("quejas-clientes.csv", low_memory=False)
df.drop(columns=["Unnamed: 0"], inplace=True)

c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [3]:
df

,Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
0,1291006,Debt collection,NaN,Communication tactics,Frequent or repeated calls,TX,76119.0,2015-03-19,2015-03-19,"Premium Asset Services, LLC",In progress,Yes,NaN
1,1290580,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt is not mine,TX,77479.0,2015-03-19,2015-03-19,Accounts Receivable Consultants Inc.,Closed with explanation,Yes,NaN
2,1290564,Mortgage,FHA mortgage,"Application, originator, mortgage broker",NaN,MA,2127.0,2015-03-19,2015-03-19,RBS Citizens,Closed with explanation,Yes,Yes
3,1291615,Credit card,NaN,Other,NaN,CA,92592.0,2015-03-19,2015-03-19,Navy FCU,In progress,Yes,NaN
4,1292165,Debt collection,Non-federal student loan,Cont'd attempts collect debt not owed,Debt resulted from identity theft,NaN,43068.0,2015-03-19,2015-03-19,Transworld Systems Inc.,In progress,Yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28151,1178184,Debt collection,NaN,Cont'd attempts collect debt not owed,Debt is not mine,TX,75201.0,2015-01-01,2015-01-02,"Enhanced Recovery Company, LLC",Closed with non-monetary relief,Yes,No
28152,1178183,Consumer loan,Vehicle loan,Taking out the loan or lease,NaN,GA,30043.0,2015-01-01,2015-01-07,DriveTime,Closed with explanation,Yes,NaN
28153,1178182,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,MN,55105.0,2015-01-01,2015-01-02,Regions,Closed with non-monetary relief,Yes,No
28154,1178181,Payday loan,NaN,Can't stop charges to bank account,NaN,CA,92509.0,2015-01-01,2015-01-07,Moneytree Inc,Closed with explanation,Yes,NaN


Queremos predecir si tendra una timely response o no.
Segyun el EDA la compañia no tiene importancia, 
tampoco tiene sentido que dejemos las quejas que esten en progreso.


In [4]:
df["Consumer disputed?"].fillna("unknown", inplace=True)

C:\Users\unaif\AppData\Local\Temp\ipykernel_24376\1677353787.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Consumer disputed?"].fillna("unknown", inplace=True)


In [5]:
df["Consumer disputed?"].unique()

array(['unknown', 'Yes', 'No'], dtype=object)

In [6]:
df=df.drop(columns=["Company", "Date received",	"Date sent to company" ])
df=df[df["Company response"]!="In progress"]
df=df[df["Consumer disputed?"]!="unknown"]

In [7]:
df.drop(columns=["Complaint ID"], inplace=True)
df

,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Company response,Timely response?,Consumer disputed?
2,Mortgage,FHA mortgage,"Application, originator, mortgage broker",NaN,MA,2127.0,Closed with explanation,Yes,Yes
6,Consumer loan,Installment loan,Managing the loan or lease,NaN,OH,44241.0,Closed with explanation,Yes,Yes
9,Debt collection,Payday loan,Taking/threatening an illegal action,Threatened arrest/jail if do not pay,TX,77433.0,Closed with explanation,Yes,Yes
14,Debt collection,"Other (phone, health club, etc.)",Disclosure verification of debt,Not given enough info to verify debt,UT,84405.0,Closed with non-monetary relief,Yes,Yes
22,Debt collection,NaN,Improper contact or sharing of info,Contacted me after I asked not to,IL,60411.0,Closed with explanation,Yes,Yes
...,...,...,...,...,...,...,...,...,...
28149,Credit reporting,NaN,Improper use of my credit report,Report improperly shared by CRC,MI,48622.0,Closed with explanation,Yes,No
28150,Credit card,NaN,Late fee,NaN,AR,72160.0,Closed with monetary relief,Yes,No
28151,Debt collection,NaN,Cont'd attempts collect debt not owed,Debt is not mine,TX,75201.0,Closed with non-monetary relief,Yes,No
28153,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,MN,55105.0,Closed with non-monetary relief,Yes,No


In [8]:
df_processed = preprocesing_function_onehot(df)
df_processed.drop(columns=["Consumer disputed?"], inplace=True)
df_processed.head()

c:\Users\unaif\OneDrive\Documentos\Documentos_Clase_Data\PROYECTOS\nombre_proyecto_final_ML\Projecto-Final-ML\src\tool_preprocess_onehot.py:373: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["ZIP code"].fillna(000, inplace=True)
c:\Users\unaif\OneDrive\Documentos\Documentos_Clase_Data\PROYECTOS\nombre_proyecto_final_ML\Projecto-Final-ML\src\tool_preprocess_onehot.py:376: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change 

,ZIP code,Timely response?,Product_Bank account or service,Product_Consumer loan,Product_Credit card,Product_Credit reporting,Product_Debt collection,Product_Money transfers,Product_Mortgage,Product_Other financial service,...,State_TX,State_UT,State_Unknown,State_VA,State_VI,State_VT,State_WA,State_WI,State_WV,State_WY
2,2127,1,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
6,44241,1,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,77433,1,False,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,False,False
14,84405,1,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False
22,60411,1,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6006 entries, 2 to 28155
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Product             6006 non-null   object 
 1   Sub-product         3736 non-null   object 
 2   Issue               6006 non-null   object 
 3   Sub-issue           2778 non-null   object 
 4   State               5930 non-null   object 
 5   ZIP code            5963 non-null   float64
 6   Company response    6006 non-null   object 
 7   Timely response?    6006 non-null   object 
 8   Consumer disputed?  6006 non-null   object 
dtypes: float64(1), object(8)
memory usage: 598.3+ KB


In [10]:
X=df_processed.drop(columns=["Timely response?"])

In [11]:
y=df_processed["Timely response?"]

In [12]:
from imblearn.over_sampling import ADASYN

In [13]:
X_resampled, y_resampled = ADASYN().fit_resample(X, y)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = 0.20, random_state = 24)

In [15]:
X_train

,ZIP code,Product_Bank account or service,Product_Consumer loan,Product_Credit card,Product_Credit reporting,Product_Debt collection,Product_Money transfers,Product_Mortgage,Product_Other financial service,Product_Payday loan,...,State_TX,State_UT,State_Unknown,State_VA,State_VI,State_VT,State_WA,State_WI,State_WV,State_WY
4401,32159,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1357,94606,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
256,30010,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3415,33905,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
5322,80306,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5249,78701,False,False,True,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
10385,96609,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3473,22554,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
8535,91855,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [16]:
y_train.unique()

array([1, 0])

In [17]:
import optuna
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Función objetivo para optimizar
"""def objective(trial):
    # Sugerencia de hiperparámetros
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
    C = trial.suggest_loguniform('C', 1e-3, 1e2)

    if kernel in ['rbf', 'poly', 'sigmoid']:
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    else:
        gamma = 'scale'  # no se usa en 'linear', pero hay que definirlo

    if kernel == 'poly':
        degree = trial.suggest_int('degree', 2, 5)
    else:
        degree = 3  # valor por defecto

    # Crear el modelo
    model = make_pipeline(
        StandardScaler(),  # importante para SVM
        SVC(
            kernel=kernel,
            C=C,
            gamma=gamma,
            degree=degree
        )
    )

    # Validación cruzada
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    return score.mean()

# Crear y ejecutar el estudio
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Resultados
print("Mejores parámetros encontrados:")
print(study.best_params)
print(f"Mejor accuracy: {study.best_value:.4f}")"""


c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'def objective(trial):\n    # Sugerencia de hiperparámetros\n    kernel = trial.suggest_categorical(\'kernel\', [\'linear\', \'rbf\', \'poly\', \'sigmoid\'])\n    C = trial.suggest_loguniform(\'C\', 1e-3, 1e2)\n\n    if kernel in [\'rbf\', \'poly\', \'sigmoid\']:\n        gamma = trial.suggest_categorical(\'gamma\', [\'scale\', \'auto\'])\n    else:\n        gamma = \'scale\'  # no se usa en \'linear\', pero hay que definirlo\n\n    if kernel == \'poly\':\n        degree = trial.suggest_int(\'degree\', 2, 5)\n    else:\n        degree = 3  # valor por defecto\n\n    # Crear el modelo\n    model = make_pipeline(\n        StandardScaler(),  # importante para SVM\n        SVC(\n            kernel=kernel,\n            C=C,\n            gamma=gamma,\n            degree=degree\n        )\n    )\n\n    # Validación cruzada\n    score = cross_val_score(model, X_train, y_train, cv=5, scoring=\'accuracy\')\n    return score.mean()\n\n# Crear y ejecutar el estudio\nstudy = optuna.create_study(dir

Tras una sin terminar este el el mejor parametro {'kernel': 'linear', 'C': 0.2859643021815645}

In [18]:
{'kernel': 'linear', 'C': 0.2859643021815645}

{'kernel': 'linear', 'C': 0.2859643021815645}

In [19]:
from sklearn.svm import SVC

In [20]:
SVC_class=SVC(random_state=24, kernel= 'linear', C= 0.286, class_weight='balanced')

In [21]:
from sklearn.model_selection import GridSearchCV

"""parameters = [{'kernel': ['linear'], 'C': [0.01, 0.1, 1, 10, 100]},
    {'kernel': ['rbf', 'sigmoid'], 'C': [0.01, 0.1, 1, 10, 100], 'gamma': ['scale', 'auto']},
    {'kernel': ['poly'], 'C': [0.01, 0.1, 1, 10, 100], 'gamma': ['scale', 'auto'], 'degree': [2, 3, 4, 5]}]



svc_class = GridSearchCV(estimator = SVC_class1,
                  param_grid = parameters,
                  n_jobs = -1,
                  cv = 10,
                  scoring="accuracy", 
                  refit=True)"""

'parameters = [{\'kernel\': [\'linear\'], \'C\': [0.01, 0.1, 1, 10, 100]},\n    {\'kernel\': [\'rbf\', \'sigmoid\'], \'C\': [0.01, 0.1, 1, 10, 100], \'gamma\': [\'scale\', \'auto\']},\n    {\'kernel\': [\'poly\'], \'C\': [0.01, 0.1, 1, 10, 100], \'gamma\': [\'scale\', \'auto\'], \'degree\': [2, 3, 4, 5]}]\n\n\n\nsvc_class = GridSearchCV(estimator = SVC_class1,\n                  param_grid = parameters,\n                  n_jobs = -1,\n                  cv = 10,\n                  scoring="accuracy", \n                  refit=True)'

In [ ]:
SVC_class.fit(X_train, y_train)

In [ ]:
"""results = pd.DataFrame(svc_class.cv_results_)
results"""

'results = pd.DataFrame(svc_class.cv_results_)\nresults'

In [ ]:
"""
results = pd.DataFrame(svc_class.cv_results_)
results
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 5))
plt.plot(results['param_n_neighbors'], results['mean_test_score'],  label='Validation Accuracy')
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. N neighbour (Grid Search)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
"""

"\nresults = pd.DataFrame(svc_class.cv_results_)\nresults\nimport matplotlib.pyplot as plt\nplt.figure(figsize=(8, 5))\nplt.plot(results['param_n_neighbors'], results['mean_test_score'],  label='Validation Accuracy')\nplt.xlabel('max_depth')\nplt.ylabel('Accuracy')\nplt.title('Accuracy vs. N neighbour (Grid Search)')\nplt.legend()\nplt.grid(True)\nplt.tight_layout()\nplt.show()\n"

In [ ]:
"""svc_class_def=SVC(**svc_class.best_params_)"""

'svc_class_def=SVC(**svc_class.best_params_)'

In [ ]:
"""svc_class_def.fit(X_train, y_train)"""

'svc_class_def.fit(X_train, y_train)'

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = (SVC_class.predict(X_test) >= 0.4).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred, normalize='true'))


NameError: name 'SVC_class' is not defined

In [ ]:
from sklearn.model_selection import cross_validate
cross_val=cross_validate(SVC_class, X_train, y_train, cv=10)["test_score"]

print(cross_val.mean())
print(cross_val.std())

In [ ]:
import os 
import pickle
os.chdir("../models")

with open('modelo_timely_SVC.pkl', 'wb') as f:
    pickle.dump(svc_class_def, f)